In [1]:
import torch
import torchvision
from model import Autoencoder
from train_dir import train
from torchvision import transforms
from torch import optim,nn

c:\Users\91963\.conda\envs\pytorch-G\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt

In [3]:
device = 'cuda:0' if torch.cuda.is_available( ) else 'cpu'
train_size = 0.9
lr = 1e-3
eps = 1e-8
batch_size = 256
epochs = 30

In [4]:
def mnist(train_part , transform= None):
    dataset = torchvision.datasets.MNIST('.',download=True,transform= transform)
    train_part = int(train_part*len(dataset))
    train_dataset,test_dataset = torch.utils.data.random_split(dataset,[train_part,len(dataset)-train_part])
    return train_dataset,test_dataset

In [5]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset , test_dataset = mnist(train_size,transform)
train_dataLoader = torch.utils.data.DataLoader(train_dataset,drop_last=True,batch_size=batch_size,shuffle=True)
test_dataLoader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=False)# why we need batch size 1 for test data set
dataLoaders = (train_dataLoader,test_dataLoader)

In [6]:
def plotn(n,data,noisy=False,super_res =None):
    fig,ax = plt.subplots(1,n)
    for i,z in enumerate(data):
        if i ==n:
            break
        

In [7]:
model = Autoencoder().to(device)
optim = optim.Adam(model.parameters(),lr =lr,eps =eps)
loss_fn = nn.BCELoss()

In [8]:
train(dataLoaders,model,loss_fn,optim,epochs,device)

  0%|          | 0/30 [00:00<?, ?it/s]


ValueError: Using a target size (torch.Size([256, 1, 28, 28])) that is different to the input size (torch.Size([256, 1, 24, 24])) is deprecated. Please ensure they have the same size.